In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
path = '/gdrive/My Drive/Lemalabs/Data/'
data = pd.read_csv(path + 'paint_mnist.csv', header=None)

***EXPLORATION OF DATASET AND DATA_PREPROCESSING***

In [4]:
data.head() #To explore the Dataset

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,255,255,255,255,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
data.shape

(3599, 785)

In [6]:
cols=data.shape[1]

x=np.array(data.iloc[:,1:cols])
y=np.array(data.iloc[:,:1])

In [7]:
x.shape, y.shape

((3599, 784), (3599, 1))

In [8]:
b= np.unique(y)
b

array([0, 1, 2, 3])

In [9]:
classes=b.size
classes

4

In [10]:
from sklearn.model_selection import train_test_split

x_train, x_test,y_train, y_test= train_test_split(x,y, test_size=0.2, random_state=2)

x_train.shape, x_test.shape, y_train.shape, y_test.shape

((2879, 784), (720, 784), (2879, 1), (720, 1))

In [11]:
x= np.array(x_train)
y= np.array(y_train)

In [12]:
rows= x.shape[0]
x=np.insert(x,0,values=np.ones(rows),axis=1)
x.shape

(2879, 785)

In [13]:
params=x.shape[1]
all_theta=np.zeros((classes,params))
all_theta.shape

(4, 785)

***MODEL OR FUNCTION MAPPING***

In [14]:
def sigmoid(z):
  return 1/(1+ np.exp(-z))

In [15]:
def computeCost(theta, x, y):
  theta = np.matrix(theta)
  x=np.matrix(x)
  y=np.matrix(y)

  first= np.multiply(-y,np.log(sigmoid(x*theta.T)))

  second= np.multiply((1-y),np.log(1-sigmoid(x*theta.T)))

  cost=np.sum(first-second)
  cost=cost/(2*len(x))

  return cost

In [16]:
cost = computeCost(all_theta,x,y)
print(cost)

1.3862943611198906


In [17]:
learning_rate=2
def gradient(theta,x,y):
  theta = np.matrix(theta)
  x=np.matrix(x)
  y=np.matrix(y)

  error= sigmoid(x*theta.T)-y
  grad= ((x.T*error)/len(x)) * learning_rate
  return grad 

In [18]:
from scipy.optimize import minimize
def one_vs_all(x,y,classes,new_theta):
    params = x.shape[1]
    rows = x.shape[0]

    for i in range(0,classes):
      theta = new_theta[i,:]

      y_i = np.array([1 if label ==i else 0 for label in y])
      y_i = np.reshape(y_i,(rows,1))

        #minimize the cost function --> Gradient descent
      fmin = minimize(fun=computeCost, x0=theta, args=(x,y_i), method='TNC', jac=gradient)
      all_theta[i,:]= fmin.x

      print(fmin.fun)

    return all_theta

In [19]:
 new_theta = one_vs_all(x,y,classes,all_theta)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in multiply
  


0.030584462811739994
0.19419626403119944
0.1733048908868248
0.037195226187325105


In [20]:
x.shape, y.shape

((2879, 785), (2879, 1))

In [ ]:
def predict_all(x, theta):

  x = np.matrix(x)
  theta = np.matrix(theta)

  h= sigmoid(x* theta.T)

  h_argmax = np.argmax(h, axis=1)

  return h_argmax,h 

In [ ]:
y_pred, h = predict_all(x, new_theta)
print(y_pred)

[[2]
 [3]
 [1]
 ...
 [0]
 [1]
 [1]]


In [ ]:
def acc(y_pred,y):
  correct = [1 if a==b else 0 for (a,b) in zip(y_pred,y)]
  accuracy = (sum(correct)/ len(correct))
  return accuracy

In [ ]:
accuracy = acc(y_pred,y)
print("Training data accuracy =",accuracy*100,"%")

Training data accuracy = 97.67280305661689 %


In [ ]:
t_x = np.array(x_test)
t_y = np.array(y_test)

t_rows=t_x.shape[0]
t_x= np.insert(t_x,0,values=np.ones(t_rows), axis=1)

In [ ]:
y_pred1, h = predict_all(t_x, new_theta)
print(y_pred1.shape)

(720, 1)


In [ ]:
accuracy1 = acc(y_pred1,t_y)
print("Test data accuracy =",(accuracy1*100),"%")

Test data accuracy = 77.22222222222223 %


In [ ]:
pic_path="/gdrive/My Drive/Lemalabs/Participants/Revanth Krishna/Week_2/"

In [ ]:
from PIL import Image
from PIL import ImageOps
from IPython.display import display

Im = Image.open(pic_path+ 'revanth_pic.png')
display(Im)

In [ ]:
Im = Im.convert('L') #L converts to gray scale
Im = ImageOps.invert(Im)
display(Im)

In [ ]:
Im = np.array(Im)
Im = np.ravel(Im) #flatten to 1 row
Im.shape

(784,)

In [ ]:
Im = Im.reshape(1,-1)

input_x = np.insert(Im, 0, 1, axis =1)

In [ ]:
input_x.shape

(1, 785)

In [ ]:
pred,h = predict_all(input_x, new_theta)

print("confidence matrix = {}".format(h))

print("predicted class is : {}",format(pred.item()))

print("confidence = {}%".format(((h[0,pred])*100).item()))

confidence matrix = [[1.29662528e-05 2.84665081e-05 9.99946204e-01 8.94565994e-05]]
predicted class is : {} 2
confidence = 99.99462035914193%
